In [16]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [17]:
import pandas as pd

In [18]:
patch_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_data_patches/'

In [19]:
for stack in ['MD594']:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:

    print stack

    box_x, box_y, box_w, box_h = detect_bbox_lookup[stack]
    first, last = detect_bbox_range_lookup[stack]

    patch_size = 224
    half_size = patch_size/2
    stride = 56

    dm = DataManager(stack=stack)

    ys, xs = np.meshgrid(np.arange(half_size, dm.image_height - half_size, stride), 
                         np.arange(half_size, dm.image_width - half_size, stride),
                         indexing='xy')

    sample_locations = np.c_[xs.flat, ys.flat]


    xmin = max(half_size, box_x*32)
    xmax = min(dm.image_width-half_size-1, (box_x+box_w)*32)
    ymin = max(half_size, box_y*32)
    ymax = min(dm.image_height-half_size-1, (box_y+box_h)*32)

    ######################

    indices_roi_allSections = defaultdict(dict)

    for sec in range(first, last+1):

        dm.set_slice(sec)

        mask = dm.load_thumbnail_mask()
        indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
        indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

        indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                            sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

        indices_roi = np.setdiff1d(indices_roi, indices_bg)
        print len(indices_roi), 'patches in ROI'

        indices_roi_allSections[sec]['roi1'] = indices_roi


    indices_roi_allSections_df = pd.DataFrame(indices_roi_allSections)

    # indices_roi_allSections_df

    dump_path = os.path.join(patch_dir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})

    indices_roi_allSections_df.to_hdf(dump_path, 'indices_allROIs_allSections')
    pd.Series([patch_size, stride, dm.image_width, dm.image_height]).to_hdf(dump_path, 'grid_parameters')

MD594
14162 patches in ROI
14390 patches in ROI
14609 patches in ROI
14715 patches in ROI
15101 patches in ROI
15084 patches in ROI
15466 patches in ROI
15625 patches in ROI
15694 patches in ROI
16071 patches in ROI
16147 patches in ROI
16184 patches in ROI
16656 patches in ROI
16768 patches in ROI
16938 patches in ROI
16892 patches in ROI
16857 patches in ROI
17094 patches in ROI
17246 patches in ROI
17394 patches in ROI
17496 patches in ROI
17666 patches in ROI
17741 patches in ROI
17928 patches in ROI
17887 patches in ROI
18046 patches in ROI
18345 patches in ROI
18328 patches in ROI
18318 patches in ROI
18315 patches in ROI
18418 patches in ROI
18484 patches in ROI
18448 patches in ROI
18725 patches in ROI
18727 patches in ROI
18748 patches in ROI
18771 patches in ROI
19050 patches in ROI
19025 patches in ROI
19052 patches in ROI
18924 patches in ROI
19057 patches in ROI
19042 patches in ROI
19085 patches in ROI
18274 patches in ROI
19247 patches in ROI
19194 patches in ROI
19444 p

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 2